## Task 1 ##

Есть файл с именами (https://yadi.sk/i/97rbNP2ucGoAKw). Нужно выполнить следующие действия и посчитать результат:

1. Отсортировать все имена в лексикографическом порядке
2. Посчитать для каждого имени алфавитную сумму – сумму порядковых номеров букв (MAY: 13 + 1 + 25 = 39)
3. Умножить алфавитную сумму каждого имени на порядковый номер имени в отсортированном списке (индексация начинается с 1). Например, если MAY находится на 63 месте в списке, то результат для него будет 63 * 39 = 2457.
4. Просуммировать произведения из п. 3 для всех имен из файла и получить число. Это число и будет ответом.

In [78]:
with open('names.txt', 'r') as f:
    names = f.read().split('","')

# Remove quotes from first and last strings
names[0] = names[0].lstrip('"')
names[-1] = names[-1].rstrip('"')

names.sort()

# Get alphabet sums
sums = [sum([ord(w) - 64 for w in name]) for name in names]

# Mupltiply by position in list
values = [i * s for i, s in enumerate(sums, 1)]

print(sum(values))

871853874


## Task 2 ##

Есть файл со строками (https://yadi.sk/d/u6AOqGqKozTrpw) вида :

```<host>\t<ip>\t<page>\n```

Нужно вывести 5 айпи-адресов, которые встречаются чаще других.

In [137]:
from collections import defaultdict

# Count ips in dict with deafult value 0
counts = defaultdict(int)

with open('hits.txt') as f:
    # Read file line by line
    line = f.readline()
    
    while line:
        ip = line.split()[1]
        counts[ip] += 1
        line = f.readline()
       
# Sort ips by count
sorted_counts = {ip: count for ip, count in sorted(counts.items(), key=lambda item: item[1], reverse=True)}
most_popular = list(sorted_counts.keys())[:5]
for item in list(sorted_counts.items())[:5]:
    print(f'{item[0]}: {item[1]}')

154.157.157.156: 1531
82.146.232.163: 1505
194.78.107.33: 1494
226.247.119.128: 1494
21.143.243.182: 1479


## Task 3 ##

В нашей школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информацию о каждом уроке мы старательно записываем в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик (существуют еще тренировочные уроки, у которых единственный участник – репетитор, но для нашей задачи они неинтересны). В конце урока, когда учитель и ученик покидают класс, они могут оценить (а могут и пропустить этот этап) качество пройденного урока по пятибальной шкале. Все оценки мы записываем в таблицу quality, чтобы мы могли узнавать, хорошо ли работает наш сайт.

В этой задаче тебе нужно будет в каждый день найти репетитора, уроки которого получили самую низкую оценку качества, и вывести эту оценку.

В архиве (https://yadi.sk/d/ESH_lU5sZRdKhA) содержится четыре файла с выгрузкой строк из базы.

lessons.txt содержит следующие поля:
id – уникальный идентификатор урока.
event_id – идентификатор, связывающий уроки с файлом participants. У нескольких уроков может быть один и тот же event_id. Подробнее про это будет позже
subject – предмет урока, просто строка
scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды). Время указано в UTC.

quality.txt содержит следующие поля:
lesson_id – идентификатор урока, указывает на запись в таблице lessons.txt. У нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (например, оба участника, ученик и репетитор могут оценить качество урока)
tech_quality – собственно оценка качества урока. Это число от 1 до 5. Иногда его может не быть, если пользователь не выставил оценку.

users.txt хранит информацию о пользователях и содержит всего два поля:
id – уникальный идентификатор пользователя
role – указывает является ли пользователь учеником (pupil) или учителем (tutor)

Файл participants.txt позволяет связать урок с его участниками. Он содержит следующие поля:
user_id – идентификатор пользователя (указывает на запись в файле users.txt)
event_id – идентификатор, связывающий урок с участником. То есть, чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х.

Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).
2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.
3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).
4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой. То есть как-то так:
2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67
2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25
...
4. Учитывать нужно только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка. Если у нескольких учителей одна и та же самая оценка, можно вывести любого.

In [96]:
'''
Load data in given format.
For each line returns list of parsed string values
'''
def readfile(filename):
    with open(filename, 'r') as f:
        # We don't need headers and lines count
        data = f.readlines()[2:-2]
    
    # Now we can clear data
    result = []
    
    for line in data:
        line_data = line.replace(' ', '').rstrip('\n').split('|')
        result.append(line_data)
        
    return result

In [135]:
from collections import defaultdict

# Load data
dir = 'tech_quality/'
participants = defaultdict(list)
    
# Save to dict lessons with subject == 'phys'
lessons_by_date = defaultdict(list)
data = readfile(dir + 'lessons.txt')
for line_data in data:
    if line_data[2] == 'phys':
        l_id = line_data[0]
        l_event = line_data[1]
        l_date = line_data[3][:10]
        lessons_by_date[l_date].append({'event': l_event, 'id': l_id})
            
# Associate lesson id with it's quality values
quality_by_lesson = defaultdict(list)
data = readfile(dir + 'quality.txt')
for line_data in data:
    # Check if there is quality value
    if line_data[1]:
        lesson_id, lesson_quality = line_data
        quality_by_lesson[lesson_id].append(int(lesson_quality))

# Get tutors list
tutors = []
data = readfile(dir + 'users.txt')
for line_data in data:
    user_id, role = line_data
    if role == 'tutor' and user_id not in tutors:
        tutors.append(user_id)
        
# Associate lesson id with tutor
tutors_by_event = {}
data = readfile(dir + 'participants.txt')
for line_data in data:
    event, user = line_data
    if user in tutors:
        tutors_by_event[event] = user
        
# Now we can find the answer
for date in sorted(lessons_by_date):
    quality_by_tutor = defaultdict(list)
    lessons_count = defaultdict(int)
    
    for lesson in lessons_by_date[date]:
        tutor = tutors_by_event[lesson['event']]
        quality = quality_by_lesson[lesson['id']]
        quality_by_tutor[tutor].extend(quality)
        lessons_count[tutor] += 1
    
    # Total score for each tutor who has at least 1 mark at this date
    score_by_tutor = [(tutor, sum(quality_by_tutor[tutor])/lessons_count[tutor]) 
                      for tutor, quality in quality_by_tutor.items()
                         if quality_by_tutor[tutor]]
    
    min_tutor, min_score = min(score_by_tutor, key=lambda v: v[1])
    print(f'{date} {min_tutor} {min_score:.2f}')

2020-01-11 c0c073c8-cb9c-4ebc-ac13-472c06029591 1.67
2020-01-12 b37ccae8-fc31-4ad8-8f55-ca855b23fbf6 1.67
2020-01-13 2fa2ab62-f1b0-4036-872f-bcfd9a8686ff 5.00
2020-01-14 43efce48-94b2-4412-857f-223d45969008 3.33
2020-01-15 603b8641-c6f6-4d89-ac89-88e50d45aa0d 5.00
2020-01-16 696c838e-c054-4e9f-a51a-50bf5660f364 2.50
2020-01-17 43efce48-94b2-4412-857f-223d45969008 3.50
2020-01-18 c0c073c8-cb9c-4ebc-ac13-472c06029591 2.50
2020-01-19 4df2832a-1d63-4453-9659-43993fc35996 1.67
2020-01-20 696c838e-c054-4e9f-a51a-50bf5660f364 5.00
